# weibo 크롤링

## 크롤링 조건 설정

In [1]:
# 페이지 스크롤 다운 횟수 설정(숫자가 클수록 많은 페이지 크롤링)
num_pagedown = 2
# 날짜 설정 : "설정 날짜" 포함, 이후 날짜 데이터 출력됨.
start_date = '2022-01-01'

In [2]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
import pandas as pd
from datetime import datetime

In [3]:
def crawling(num_scrolldown, start_date):

    url = 'https://weibo.com/p/1001066011831795/home?from=page_100106&mod=TAB#place'
    # webdriver 로드
    driver.get(url)    
    time.sleep(5)
    driver.implicitly_wait(5)

    # 페이지 스크롤 다운
    for i in range(num_scrolldown):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        driver.implicitly_wait(5)

    # 페이지 크롤링
    soup = bs(driver.page_source, 'lxml')
    print("타이틀 : ", soup.title.text)

    # 타이틀 수집
    raw_contents = soup.find_all('div', class_='WB_text W_f14')
    content_list = []
    num = len(raw_contents)
    for i in range(num):
        content = raw_contents[i].text.strip()
        content_list.append(content)
    print('원본 게시글 수 : ', len(raw_contents))
    print('게시글 수 : ', len(content_list))

    # 조회수 수집
    raw_number_view = soup.find_all('i', class_='S_txt2')
    number_view_list = []
    for i in range(len(raw_number_view)):
        temp = raw_number_view[i].attrs['title']
        number_view = int(temp[9:-1])
        number_view_list.append(number_view)
    print('조회수 리스트 원소수 : ', len(number_view_list))

    # 정보 수집
    raw_info = soup.find_all('span', 'line S_line1')
    # 좋아요 수 수집
    number_like_list = []
    num_range = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range):
        num = ( i + 1 ) * 4 - 1
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        number_like = int(temp)
        number_like_list.append(number_like)
    print('좋아요 리스트 원소수 : ', len(number_like_list))

    # 공유 수 수집
    number_share_list = []
    num_range = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range):
        num = ( i + 1 ) * 4 - 3
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        if '转发' in temp:
            number_share_list.append(0)
        else:
            number_like = int(temp)
            number_share_list.append(number_like)
    print('공유수 리스트 원소수 : ', len(number_share_list))

    # 댓글수 수집
    number_comment_list = []
    num_range_like = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range_like):
        num = ( i + 1 ) * 4 - 2
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        if '评论' in temp:
            number_comment_list.append(0)
        else:
            number_like = int(temp)
            number_comment_list.append(number_like)
    print('댓글수 리스트 원소수 : ', len(number_comment_list))

    # 게시글 날짜 수집
    date_list = []
    raw_date = soup.find_all('div', 'WB_from S_txt2')
    for i in range(len(raw_date)):
        temp = raw_date[i].find('a', 'S_txt2')
        date = temp.attrs['title'][:10]
        date_list.append(date)
    print('날짜 리스트 원소수 : ', len(date_list))

    # 팔로워 수 수집
    number_follower_list = []
    number_of_follower = soup.find_all('strong', 'W_f16')
    num = int(number_of_follower[1].text)
    number_follower_list.append(num)
    for i in range(len(date_list)-1):
        number_follower_list.append(0)
    print('팔로워 수 리스트 원소수 : ', len(number_follower_list))

    # 데이터 프레임 생성
    df = {'날짜':date_list,
        '제목':content_list,
        '조회수':number_view_list,
        '좋아요_수':number_like_list,
        '공유수':number_share_list,
        '댓글수':number_comment_list,
        '팔로워_수':number_follower_list}
    data = pd.DataFrame(df)
    
    # 데이터 셋 날짜로 슬라이싱
    final_data = data[data['날짜'] >= start_date]

    # 파일로 출력
    file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
    file_name = 'weibo_' + file_name + '.xlsx'
    path = './crawling_data/weibo/'
    final_data.to_excel(path+file_name, index=False, encoding='utf-8')

    return data, final_data

In [4]:
# 사이트 로그인
driver = webdriver.Chrome('chromedriver99')
driver.get("https://weibo.com/login.php")
time.sleep(5)
driver.implicitly_wait(15)

# 창 최대화
driver.maximize_window()
time.sleep(1)

username = 'globalgg1@gg.go.kr'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
element_id.send_keys(username)
time.sleep(1)
element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
element_password.send_keys(password)

time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()
time.sleep(5)

<ipython-input-4-fd217827f503>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver99')
<ipython-input-4-fd217827f503>:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
<ipython-input-4-fd217827f503>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
<ipython-input-4-fd217827f503>:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()


In [5]:
df, final_df = crawling(num_pagedown, start_date)

타이틀 :  京畿道官博的微博_微博
원본 게시글 수 :  45
게시글 수 :  45
조회수 리스트 원소수 :  45
좋아요 리스트 원소수 :  45
공유수 리스트 원소수 :  45
댓글수 리스트 원소수 :  45
날짜 리스트 원소수 :  45
팔로워 수 리스트 원소수 :  45


# 크롤링 데이터 확인

## 슬라이싱한 크롤링 데이터

In [6]:
final_df

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수
0,2022-03-10,走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花...,1558,4,0,2,530830
1,2022-03-08,可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...,2417,1,0,0,0
2,2022-03-04,欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...,3798,1,1,0,0
3,2022-03-02,京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地...,7242,1,0,0,0
4,2022-02-28,京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...,6406,1,0,0,0
5,2022-02-24,历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线。京畿道光明市是由于首尔地区变大而产生的...,13599,3,0,0,0
6,2022-02-22,"世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市。空...",8472,2,0,0,0
7,2022-02-18,京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区。议政府是位于京畿道北部，首都的关口...,10533,2,0,1,0
8,2022-02-15,以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...,14157,2,1,0,0
9,2022-02-10,有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...,14166,4,0,0,0


## 전체 크롤링 데이터

In [7]:
df

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수
0,2022-03-10,走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花...,1558,4,0,2,530830
1,2022-03-08,可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...,2417,1,0,0,0
2,2022-03-04,欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...,3798,1,1,0,0
3,2022-03-02,京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地...,7242,1,0,0,0
4,2022-02-28,京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...,6406,1,0,0,0
5,2022-02-24,历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线。京畿道光明市是由于首尔地区变大而产生的...,13599,3,0,0,0
6,2022-02-22,"世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市。空...",8472,2,0,0,0
7,2022-02-18,京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区。议政府是位于京畿道北部，首都的关口...,10533,2,0,1,0
8,2022-02-15,以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...,14157,2,1,0,0
9,2022-02-10,有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...,14166,4,0,0,0


# 코드 마지막